In [1]:
# part 1
def process_flow(part,flow):
    i = 1
    while i<10: # still in the flow
        print(flow[i])
        if flow[i][1] in '<>':
            condition = flow[i].split(':')
            stat = 'xmas'.index(flow[i][0])
            if eval(part[stat] + condition[0][1:]):
                return condition[1]
            else:
                i += 1
        else:
            return flow[i]

# main

# input workflows
lines = open("input_19_test.txt").read().splitlines()
workflows = []
i = 0
while lines[i] != '':
    # print(f'i = {i} workflow = {lines[i]}')
    # read the workflow
    name, leftover = lines[i].split('{')
    leftover = leftover[0:-1]
    instrs = leftover.split(',')
    instrs.insert(0,name)
    workflows.append(instrs)
    i += 1    
print(workflows)

# make a list of names of the workflows
workflow_names = [x[0] for x in workflows]
print(workflow_names)

i += 1
for i in range(i,len(lines)):
    # print(f'i = {i} part = {lines[i]}')
    line = lines[i][1:-1]
    statstr = line.split(',')
    part = [0,0,0,0]
    for k in range(4):
        part[k] = statstr[k][2:]

    print(part)
    # process the part
    next_name = 'in'
    # while 0: # still processing: 
    workflow_num = workflow_names.index(next_name)
    print(workflow_num)
    output = process_flow(part,workflows[workflow_num])
    print(output)
    z




[['px', 'a<2006:qkq', 'm>2090:A', 'rfg'], ['pv', 'a>1716:R', 'A'], ['lnx', 'm>1548:A', 'A'], ['rfg', 's<537:gd', 'x>2440:R', 'A'], ['qs', 's>3448:A', 'lnx'], ['qkq', 'x<1416:A', 'crn'], ['crn', 'x>2662:A', 'R'], ['in', 's<1351:px', 'qqz'], ['qqz', 's>2770:qs', 'm<1801:hdj', 'R'], ['gd', 'a>3333:R', 'R'], ['hdj', 'm>838:A', 'pv']]
['px', 'pv', 'lnx', 'rfg', 'qs', 'qkq', 'crn', 'in', 'qqz', 'gd', 'hdj']


['787', '2655', '1222', '2876']
7
s<1351:px


KeyboardInterrupt: 

In [64]:
a = ['c','d','e']
next = 'd'
'xmas'.index('x')
string = str(100)
print(string)
eval(string+'>30')


100


True